<h1> SEIP MIPS map homogenisation</h1>

<img src=https://avatars1.githubusercontent.com/u/7880370?s=200&v=4>

These MIPS maps have ben produced by IPAC. This code takes the individual image files and adds them into a single multi-layered fits files to have the homogeneous HELP format.  This code was adapted from code used for SPIRE homogenisaton (Yannick Roehlly) and in turn adapted for PACS data (Peter Hurely). 

<h2> Caveats</h2>
* Although many of the longer wavelength MIPS maps are available I've only run this on the MIPS 24micron maps at the moment
* Some of the SEIP maps may not have successfully down-loaded, this hasn't been checked
* only puts in image, error and coverage files (and adds blank PSF extension)

<hr>

Seb Oliver 28-March-2018

<h2> Guide</h2>
<h3>Steps: </h3>
* Obtain list of fields from sub-folders
for each field:
* read in coverage, image and noise
* attach meta data
* create PSF extension
* Save out

<h3>Things you may wish to edit</h3>
* You'll need to change the folder path in the 2nd code block to point you where the files are in your system
* Add in a specifc field (or switch to wild chracter) in the path to pick field(s)
* change the 1.mosaic pattern to 2.mosaic for MIPS 70 micron or 3.mosaic for MIPS 160.
* read in more of the available files and add as additional extensions

# Importing libraries

In [1]:
from datetime import datetime
from itertools import product
import glob

import numpy as np
import astropy as astropy

from astropy.io import fits
from astropy.table import Table
from astropy.table import Column
from astropy import wcs
VERSION = "0.9"

# This is where you set up the field and folders to work on

In [2]:
field_specified='HDF-N' # could use wild cards e.g. * would do all the fields
field_specified='XMM-LSS' # could use wild cards e.g. * would do all the fields

mainfolder='/Volumes/Seagate Backup Plus Drive/HELP/mips_fields'
moc_dir='/Volumes/Seagate Backup Plus Drive/HELP/dmu_products/dmu2/dmu2_field_coverages/'

mainfolder='/Users/sjo/Data/SEIP/mips_fields'
moc_dir='/Users/sjo/Data/dmu2_field_coverages/'

mainfolder='../../dmu17/SEIP/'
moc_dir=   '../../dmu2/dmu2_field_coverages/'

# finding all the SEIP mosaic files in the folder

In [3]:
# finding all the SEIP mosaic files (other files should be there if the mosaic file is)
filelist= glob.glob(f"{mainfolder}/{field_specified}/data/*1.mosaic.fits", recursive=True)
print("Number of files found:", len(filelist))

Number of files found: 70


# opening the table which logs the SEIP files

In [4]:
# opening the table of SEIP in which we will update some records
seip_table = fits.open(f"{mainfolder}/{field_specified}/mosaic_tiles_{field_specified}.fits", mode='update')
seip_table.info()
seip_hd=seip_table[1].header
seip_data=seip_table[1].data
#print(repr(seip_hd))
seip_table2=Table.read(f"{mainfolder}/{field_specified}/mosaic_tiles_{field_specified}.fits")
seip_table2['areahelp']=np.zeros(len(seip_table2['EXPTIME']))
#t.add_column(aa, index=0) 


Filename: ../../dmu17/SEIP//XMM-LSS/mosaic_tiles_XMM-LSS.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  mosaic_tiles.cat    1 BinTableHDU    104   447R x 37C   [J, 20A, 4A, I, J, D, D, D, D, D, D, D, I, J, I, I, I, I, 22A, 29A, D, D, D, D, D, D, D, D, D, D, E, D, 7A, L, 143A, 44A, L]   


# MOC libraries
This next block of code comes from Peter Hurley's XID libraray and allows us to create a MOC from the error array

In [5]:
import pymoc as pymoc
from pymoc import MOC
from pymoc import io

from healpy import pixelfunc
from pymoc.util import catalog

import numpy as np
def get_HEALPix_pixels(order,ra,dec,unique=True):


    """Work out what HEALPix a source is in
    :param order: order of HEALPix
    :param ra: Right Ascension
    :param dec: Declination
    :param unique: if unique is true, removes duplicate pixels
    :return: list of HEALPix pixels :rtype:
    """
    HPX_D2R=np.pi/180.0
    #Convert catalogue to polar co-ords in radians
    phi = ra*HPX_D2R
    theta = np.pi/2.0 - dec*HPX_D2R
    #calculate what pixel each object is in
    ipix = pixelfunc.ang2pix(2**order, theta, phi, nest=True)
    #return unique pixels (i.e. remove duplicates)
    if unique is True:
        return np.unique(ipix)
    else:
        return ipix


def create_MOC_from_map(good,wcs):
    """Generate MOC from map
    :param good: boolean array associated with map
    :param wcs: wcs information
    :return: MOC :rtype: pymoc.MOC
    """
    x_pix,y_pix=np.meshgrid(np.arange(0,wcs._naxis1),np.arange(0,wcs._naxis2))
    ra,dec= wcs.wcs_pix2world(x_pix,y_pix,0)

    pixels=get_HEALPix_pixels(15,ra[good],dec[good])
    map_moc=MOC()
    map_moc.add(15,pixels)
    return map_moc

# setting up global MOCS

In [6]:
# sum of all SEIP MOCS
total_moc=MOC()

# Performing the homogenisation
i.e. merging component files into a single multi-extension file and adding some keywords
traps errors so that it can continue, e.g. if a file is missing
skips over files if they've already been logged as "downloaded" (this could be improved to be an option

In [7]:
for i, file in enumerate(filelist):
    print('----------------')
    print(i,file)
    folders=file.split('/')
    print(folders[len(folders)-3])
    field=folders[len(folders)-3]
    filename=folders[len(folders)-1]
    seip_id=(filename==seip_data.field('file')) 
    print(filename,seip_data.field('file')[seip_id])
# actual HELP SPIRE MOC
    print(f'{moc_dir}{field}_MOC.fits')
#    help_moc_hdu = fits.open(f'{moc_dir}{field}_MOC.fits')
    help_moc=MOC()
#    pymoc.io.fits.read_moc_fits(help_moc, f'{moc_dir}{field}_MOC.fits',include_meta=False)
    help_moc.read(f'{moc_dir}{field}_MOC.fits')

    if seip_data.field('downloaded')[seip_id]:
        print("Already downloaded skipping file",filename)
    else:
        ext=0
    
        try:

            # Load in image map
            image_map=fits.open(file)
            image_hdu = fits.ImageHDU(header=image_map[ext].header,
                                data=image_map[ext].data)
            image_hdu.header['EXTNAME'] = "IMAGE"
 
            # Load in uncertainty map
            noise_map=fits.open(file.replace('mosaic','unc'))
            noise_hdu = fits.ImageHDU(header=noise_map[ext].header,
                                data=noise_map[ext].data)
            noise_hdu.header['EXTNAME'] = "ERROR"
 
            # we'll use the uncertainty map to generate a MOC
            good=~np.isnan(noise_map[ext].data)
            w = wcs.WCS(noise_map[ext].header)     
            m=create_MOC_from_map(good,w)
            total_moc=total_moc+m
        
            intersect=m.intersection(help_moc)
            print('Area',repr(intersect.area))
        
#        moc_hdu=io.fits.write_moc_fits_hdu(m)
#        io.fits.write_moc_fits(m,file.replace('mosaic','moc3'),overwrite=True)
        
            m.write(file.replace('mosaic','moc'), overwrite=True)
        
            # Load in coverage map
            cov_map=fits.open(file.replace('mosaic','cov'))
            cov_hdu = fits.ImageHDU(header=cov_map[ext].header,
                                data=cov_map[ext].data)
            cov_hdu.header['EXTNAME'] = "COVERAGE"

            # Mark this tile as successfully downloaded
        
            seip_data.field('downloaded')[seip_id]=True
            print('downloaded:',seip_data.field('downloaded')[seip_id])

# ----------------------------
            # mark this tile with area of overlap with HELP
            seip_data.field('inhelp')[seip_id]=(intersect.area>0.)
            seip_table2.field('areahelp')[seip_id]=(intersect.area)
            print('updating intersection with HELP')
            #print(seip_id) 
# ----------------------------

        except:
            # update the SEIP file to clearly indicate that this file is not there
            seip_data.field('downloaded')[seip_id]=False
            print,'error'
#            stop
    #fits.close(file)
        pass
            
            #Create primary header
        primary_hdu = fits.PrimaryHDU()
        primary_hdu.header.append((
            "CREATOR", "Herschel Extragalactic Legacy Project"
        ))
        primary_hdu.header.append((
            "TIMESYS", "UTC", "All dates are in UTC time"
        ))
        primary_hdu.header.append((
            "DATE", datetime.now().replace(microsecond=0).isoformat(),
            "Date of file creation"
        ))
        primary_hdu.header.append((
        "VERSION", VERSION, "HELP product version"
        ))
        primary_hdu.header.append((
        "TELESCOP", "Spitzer", "Name of the telescope"
        ))
        primary_hdu.header.append((
        "INSTRUME", "MIPS", "Name of the instrument"
        ))
        primary_hdu.header.append((
        "FILTER", "MIPS-24", "Name of the filter"
        ))
        primary_hdu.header.append((
        "FIELD", field, "Name of the HELP field"
        ))


        #Add empty extension for PSF
        psf_hdu = fits.ImageHDU()
        psf_hdu.header['EXTNAME'] = "PSF"
        psf_hdu.header.add_comment("The PSF is not available.")

        hdu_list = fits.HDUList([primary_hdu, image_hdu, noise_hdu, cov_hdu, psf_hdu])
        hdu_list.writeto(file.replace('mosaic','help'),
                 checksum=True, overwrite=True)
       # print(f"{file} /  processed...")

----------------
0 ../../dmu17/SEIP//XMM-LSS/data/60095871.60095871-67.MIPS.1.mosaic.fits
XMM-LSS
60095871.60095871-67.MIPS.1.mosaic.fits ['60095871.60095871-67.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/XMM-LSS_MOC.fits
Already downloaded skipping file 60095871.60095871-67.MIPS.1.mosaic.fits
----------------
1 ../../dmu17/SEIP//XMM-LSS/data/60095871.60095871-6.MIPS.1.mosaic.fits
XMM-LSS
60095871.60095871-6.MIPS.1.mosaic.fits ['60095871.60095871-6.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/XMM-LSS_MOC.fits
Already downloaded skipping file 60095871.60095871-6.MIPS.1.mosaic.fits
----------------
2 ../../dmu17/SEIP//XMM-LSS/data/60095871.60095871-14.MIPS.1.mosaic.fits
XMM-LSS
60095871.60095871-14.MIPS.1.mosaic.fits ['60095871.60095871-14.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/XMM-LSS_MOC.fits
Already downloaded skipping file 60095871.60095871-14.MIPS.1.mosaic.fits
----------------
3 ../../dmu17/SEIP//XMM-LSS/data/60095871.60095871-19.MIPS.1.mosaic.fits
XM

Already downloaded skipping file 60095871.60095871-51.MIPS.1.mosaic.fits
----------------
29 ../../dmu17/SEIP//XMM-LSS/data/60095871.60095871-32.MIPS.1.mosaic.fits
XMM-LSS
60095871.60095871-32.MIPS.1.mosaic.fits ['60095871.60095871-32.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/XMM-LSS_MOC.fits
Already downloaded skipping file 60095871.60095871-32.MIPS.1.mosaic.fits
----------------
30 ../../dmu17/SEIP//XMM-LSS/data/40032010.40032010-0.MIPS.1.mosaic.fits
XMM-LSS
40032010.40032010-0.MIPS.1.mosaic.fits ['40032010.40032010-0.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/XMM-LSS_MOC.fits
Already downloaded skipping file 40032010.40032010-0.MIPS.1.mosaic.fits
----------------
31 ../../dmu17/SEIP//XMM-LSS/data/60095871.60095871-61.MIPS.1.mosaic.fits
XMM-LSS
60095871.60095871-61.MIPS.1.mosaic.fits ['60095871.60095871-61.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/XMM-LSS_MOC.fits
Already downloaded skipping file 60095871.60095871-61.MIPS.1.mosaic.fits
----------------

Already downloaded skipping file 60095871.60095871-9.MIPS.1.mosaic.fits
----------------
59 ../../dmu17/SEIP//XMM-LSS/data/60095871.60095871-75.MIPS.1.mosaic.fits
XMM-LSS
60095871.60095871-75.MIPS.1.mosaic.fits ['60095871.60095871-75.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/XMM-LSS_MOC.fits
Already downloaded skipping file 60095871.60095871-75.MIPS.1.mosaic.fits
----------------
60 ../../dmu17/SEIP//XMM-LSS/data/60095871.60095871-4.MIPS.1.mosaic.fits
XMM-LSS
60095871.60095871-4.MIPS.1.mosaic.fits ['60095871.60095871-4.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/XMM-LSS_MOC.fits
Already downloaded skipping file 60095871.60095871-4.MIPS.1.mosaic.fits
----------------
61 ../../dmu17/SEIP//XMM-LSS/data/60095871.60095871-76.MIPS.1.mosaic.fits
XMM-LSS
60095871.60095871-76.MIPS.1.mosaic.fits ['60095871.60095871-76.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/XMM-LSS_MOC.fits
Already downloaded skipping file 60095871.60095871-76.MIPS.1.mosaic.fits
----------------


# Wrapping up some output files

In [8]:
# writing out the SEIP Table
seip_table.flush()
seip_table2.write(f"{mainfolder}/{field_specified}/mosaic_tiles_{field_specified}_new.fits", overwrite=True)

# writing out the combined moc
total_moc.write('help_moc.fits', overwrite=True)

<h1> Finding MOC files and writing an Aladin Script for plotting then up </h1>

In [9]:
# writing a file for plotting up MOCS in aladin

moc_filelist= glob.glob(f"{mainfolder}/{field_specified}/data/*1.moc.fits", recursive=True)
print("Number of MOC files found:", len(moc_filelist))


f=open(f"{mainfolder}/{field_specified}/aladin_script_{field_specified}.txt", 'w')
f.write("# This script can be loaded into Aladin using the Tools>Script Console (F5)\n")
f.write("# you will need to alter the file paths to be correct relative to your Aladin installation\n")
f.write("load "+moc_dir+field+"_MOC.fits \n")
#    help_moc_hdu = fits.open(f'{moc_dir}{field}_MOC.fits')

for i, file in enumerate(moc_filelist):
    f.write("load "+file.replace('mosaic','moc')+"\n")
f.close()
        

Number of MOC files found: 70
